In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd 

import torchvision 
from torchvision.transforms import transforms
from torchvision.datasets import MNIST 
from torchvision.utils import save_image, make_grid

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device {device}')

Using device cuda


In [2]:
## Hyper parameters
batch_size = 128
torch.randn(size=(12, 12)).to(device)

/home/aniket-kalra/BS/venv_bs/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GeForce GTX 970 which is of cuda capability 5.2.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/home/aniket-kalra/BS/venv_bs/lib/python3.12/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/home/aniket-kalra/BS/venv_bs/lib/python3.12/site-packages/torch/cuda/__init__.py:326: UserWarning: 
NVIDIA GeForce GTX 970 with CUDA capability sm_52 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the NVIDIA GeForce GTX 970 GPU with PyTorch, please check the instruct

AcceleratorError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])
train_data = MNIST(root='/home/aniket-kalra/BS/GenAI/data',
                    train= True, download= True, transform= transform)
train_loader = DataLoader(train_data, batch_size= batch_size, shuffle= True)

In [16]:
class BetaVAE(nn.Module):
    def __init__(self, latent_dim= 20):
        super(BetaVAE, self).__init__()
        self.encoder = nn.Sequential([
            nn.Conv2d(1, 32, kernel_size= 4, stride= 2, padding= 1), # [B, 32, 14, 14],
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size= 4, stride= 2, padding= 1),
            nn.ReLU(),
            nn.Conv2d(61, 128, kernel_size= 3, stride= 2, padding= 1), # [B, 128, 4, 4]
            nn.ReLU()
        ])
        self.flatten = nn.Flatten()
        self.fc_mu = nn.Linear(128*4*4, latent_dim)
        self.fc_logvar = nn.Linear(128*4*4, latent_dim)

        # --------- Decoder
        self.fc_decode = nn.Linear(latent_dim, 128*4*4)
        self.decoder = nn.Sequential([
            nn.ConvTranspose2d(128, 64, kernel_size= 4, stride= 2, padding= 1), #[B, 64, 8, 8]
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size= 4, stride= 2, padding= 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size= 4, stride= 2, padding= 1),
            nn.Sigmoid()
        ])

    def encode(self, x):
        h= self.encoder(x)
        h_flat = self.flatten(h)
        mu = self.fc_mu(h_flat)
        logvar = self.fc_logvar(h_flat)
        return mu, logvar 
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std 
    
    def decode(self, z):
        h = self.fc_decode(z)
        h = h.view(-1, 128, 4, 4)
        x_hat = self.decoder(h)
        return x_hat[:, :, :28, :28]
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        return x_hat, mu, logvar 


